In [34]:
# Progetto Data Science Lab
## Sentiment Analysis Corona Virus
## Giacomo
## Alessandro Pontini
## Kevin Tranchina

In [68]:
import os 
import pandas as pd
from datetime import datetime, timedelta
import nltk
import re ##regulare expression
import string
from nltk.tokenize import BlanklineTokenizer
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import subprocess
import operator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [36]:
subprocess.Popen(r'explorer /select,"C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic"')

# Import and Preprocessing Dataset

In [37]:
# imposta il path dove vuoi hai i dati

path = str(r'C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic')
path = path.replace("\\","/")
os.chdir(path)

In [38]:
# Lettura file 
#33716 righe
vaccination = pd.read_csv("vaccination_all_tweets.csv")
vaccination

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33712,1370357110693892099,🌎 Sarwar 🌐,E,W pirsarwarras,2009-07-01 00:18:11,1706,1435,6359,False,2021-03-12 12:52:49,#BREAKING \n\nThe information attack targeting...,"['BREAKING', 'SputnikV', 'US']",Twitter Web App,0,2,False
33713,1370355891170578438,CarpathianViking,"Budapest, Hungary","Carpathian Basin, the 1st populated area in Eu...",2016-08-01 14:58:38,223,2004,25374,False,2021-03-12 12:47:59,"Pity, as my personal preference would have bee...","['AstraZeneca', 'vaccine', 'Pfizer']",Twitter for Android,0,0,False
33714,1370355333529538561,Olga Semenova,Moskau,Ich bin Journalistin und kommuniziere gern Deu...,2011-07-14 14:19:40,1801,270,13732,False,2021-03-12 12:45:46,#WHO in close contact with #Russia on #Sputnik...,"['WHO', 'Russia', 'SputnikV']",Twitter Web App,0,0,False
33715,1370354789104635910,Sputnik,NaN,"Sputnik is a global wire, radio and digital ne...",2009-04-22 12:55:28,322277,306,24,True,2021-03-12 12:43:36,#URGENT | US targets Europe in smear campaign ...,['URGENT'],TweetDeck,7,19,False


In [39]:
# Preprocessing

## lowerCase
vaccination.text = vaccination.text.str.lower()

## rimozione dei numeri
vaccination.text = vaccination.text.replace(r'\d+','')

## rimozione spazi bianchi
vaccination.text  = vaccination.text.astype(str).str.strip()

# rimozione punteggiatura
# verificato rimuove anche i cancelletti
vaccination.text=vaccination.text.astype(str).str.translate(str.maketrans('','',string.punctuation))

#rimozione \n
vaccination.text = vaccination.text.replace("\n"," ")

#rimozione apice strano
vaccination.text = vaccination.text.replace("'’'"," ")

In [40]:
# TweetTokenizer
tknzr = TweetTokenizer()
vaccination['tweet_tokenize'] = vaccination['text'].apply(tknzr.tokenize)
vaccination['tweet_tokenize'] 

0        [same, folks, said, daikon, paste, could, trea...
1        [while, the, world, has, been, on, the, wrong,...
2        [coronavirus, sputnikv, astrazeneca, pfizerbio...
3        [facts, are, immutable, senator, even, when, y...
4        [explain, to, me, again, why, we, need, a, vac...
                               ...                        
33712    [breaking, the, information, attack, targeting...
33713    [pity, as, my, personal, preference, would, ha...
33714    [who, in, close, contact, with, russia, on, sp...
33715    [urgent, us, targets, europe, in, smear, campa...
33716    [breaking, us, targets, europe, in, smear, cam...
Name: tweet_tokenize, Length: 33717, dtype: object

In [41]:
# Rimozione Stop Words
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words += ['im','youre','thats','get','cant','dont','go',
               'going','gonna','like','got', 'come', 'look','see','theres',
               'say','want','wanna','make','yes','right','']


vaccination['tweet_without_stopwords'] = vaccination['tweet_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])
vaccination['tweet_without_stopwords'] 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [folks, said, daikon, paste, could, treat, cyt...
1        [world, wrong, side, history, year, hopefully,...
2        [coronavirus, sputnikv, astrazeneca, pfizerbio...
3        [facts, immutable, senator, even, ethically, s...
4        [explain, need, vaccine, borisjohnson, matthan...
                               ...                        
33712    [breaking, information, attack, targeting, spu...
33713    [pity, personal, preference, would, astrazenec...
33714    [close, contact, russia, sputnikv, certificati...
33715    [urgent, us, targets, europe, smear, campaign,...
33716    [breaking, us, targets, europe, smear, campaig...
Name: tweet_without_stopwords, Length: 33717, dtype: object

# Sentiment Analysis
## VADER ( Valence Aware Dictionary for Sentiment Reasoning)
#### VADER  is a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the NLTK package and can be applied directly to unlabeled text data.

#### To calculate the sentimental score of the entire text, Vader scans the text for known sentimental features, modified the intensity and polarity according to the rules, summed up the scores of features found within the text and normalized the final score to (-1, 1) using function:



$$ compound=\frac{x}{\sqrt{x^2 +a}}$$



#### In Vader, alpha is set to be 15 which approximates the maximum expected value of x.

In [44]:
vaccination['scores']  = vaccination['text'] .apply(lambda review: sid.polarity_scores(review))

In [57]:
vaccination['scores']

0        {'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'comp...
1        {'neg': 0.125, 'neu': 0.766, 'pos': 0.109, 'co...
2        {'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'comp...
3        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
                               ...                        
33712    {'neg': 0.162, 'neu': 0.838, 'pos': 0.0, 'comp...
33713    {'neg': 0.086, 'neu': 0.914, 'pos': 0.0, 'comp...
33714    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
33715    {'neg': 0.145, 'neu': 0.751, 'pos': 0.104, 'co...
33716    {'neg': 0.152, 'neu': 0.848, 'pos': 0.0, 'comp...
Name: scores, Length: 33717, dtype: object

In [19]:
# Test
a = 'I love my girl'
print(sid.polarity_scores(a))

[3.2, 0, 0]
I love my girl
3.2
3.2
0.6369499429264264
{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}


In [20]:
vaccination['compound']  = vaccination['scores'].apply(lambda score_dict: score_dict['compound'])
vaccination['compound']

0        0.4019
1       -0.1027
2        0.2500
3        0.0000
4        0.0000
          ...  
33712   -0.4767
33713   -0.1531
33714    0.0000
33715   -0.1779
33716   -0.3612
Name: compound, Length: 33717, dtype: float64